In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Sun Mar  2 17:38:35 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.94                 Driver Version: 560.94         CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 2060 ...  WDDM  |   00000000:26:00.0  On |                  N/A |
| 55%   57C    P2             41W /  175W |     670MiB /   8192MiB |      6%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
!pip install datasets==1.18.3
!pip install transformers==4.17.0
!pip install jiwer

  Attempting uninstall: datasets
    Found existing installation: datasets 3.3.2
    Uninstalling datasets-3.3.2:
      Successfully uninstalled datasets-3.3.2


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
evaluate 0.4.3 requires datasets>=2.0.0, but you have datasets 1.18.3 which is incompatible.


   ---------------------------------------- 0.0/3.8 MB ? eta -:--:--
   ---------------------------------------- 3.8/3.8 MB 20.7 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.50.0.dev0
    Uninstalling transformers-4.50.0.dev0:
      Successfully uninstalled transformers-4.50.0.dev0


In [3]:
! pip install git-lfs

In [6]:
! pip install soundfile

In [4]:
from huggingface_hub import notebook_login

notebook_login()

## Load Dataset

In [2]:
from datasets import Dataset, DatasetDict
import pandas as pd
import os

data_dir = os.path.abspath(os.path.join("..", "..", "data", "processed"))

# Load the dataset using pandas first, then convert to datasets
train_df = pd.read_csv(os.path.join(data_dir, "train_data.csv"))
validation_df = pd.read_csv(os.path.join(data_dir, "validation_data.csv"))
test_df = pd.read_csv(os.path.join(data_dir, "test_data.csv"))

# Convert pandas DataFrames to datasets
dataset = DatasetDict({
    "train": Dataset.from_pandas(train_df),
    "validation": Dataset.from_pandas(validation_df),
    "test": Dataset.from_pandas(test_df)
})

print(dataset)

DatasetDict({
    train: Dataset({
        features: ['text', 'path'],
        num_rows: 80
    })
    validation: Dataset({
        features: ['text', 'path'],
        num_rows: 10
    })
    test: Dataset({
        features: ['text', 'path'],
        num_rows: 10
    })
})


In [3]:
def load_audio(batch):
    # Import inside the function to ensure it's available in worker processes
    import soundfile as sf
    import numpy as np
    import os
    
    # Make sure the path is absolute
    audio_path = batch["path"]
    
    # Fix path if needed (check if the path exists)
    if not os.path.isfile(audio_path):
        # Try to fix relative path issues
        base_dir = os.path.abspath(os.path.join("..", "..", "data", "processed"))
        audio_path = os.path.join(base_dir, audio_path.replace("../processed\\", ""))
    
    # Load audio from the file path
    try:
        audio_array, sampling_rate = sf.read(audio_path)
        
        # Convert to mono if stereo
        if len(audio_array.shape) > 1:
            audio_array = audio_array.mean(axis=1)
        
        batch["audio"] = {"array": audio_array, "sampling_rate": sampling_rate}
    except Exception as e:
        print(f"Error loading audio file {audio_path}: {e}")
        # Return a default audio array to avoid failing
        batch["audio"] = {"array": np.zeros(16000), "sampling_rate": 16000}
    
    return batch

# Apply the audio loading to all splits, with fewer processes
dataset = dataset.map(load_audio, num_proc=1)  # Start with 1 process to debug

Parameter 'function'=<function load_audio at 0x00000230B340C1F0> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


0ex [00:00, ?ex/s]

0ex [00:00, ?ex/s]

0ex [00:00, ?ex/s]

## Additional Preprocessing

In [4]:
import re
chars_to_ignore_regex = '[\\,\\?\\.\\!\\-\\;\\:\\\"]'

def remove_special_characters(batch):
    batch["text"] = re.sub(chars_to_ignore_regex, '', batch["text"]).lower() + " "
    return batch

dataset = dataset.map(remove_special_characters)

0ex [00:00, ?ex/s]

0ex [00:00, ?ex/s]

0ex [00:00, ?ex/s]

In [5]:
def extract_all_chars(batch):
  all_text = " ".join(batch["text"])
  vocab = list(set(all_text))
  return {"vocab": [vocab], "all_text": [all_text]}

vocabs = dataset.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, 
                    remove_columns=dataset["train"].column_names)

vocab_list = list(set(vocabs["train"]["vocab"][0]) | set(vocabs["validation"]["vocab"][0]) | set(vocabs["test"]["vocab"][0]))

vocab_dict = {v: k for k, v in enumerate(vocab_list)}
print(f"Vocabulary size: {len(vocab_dict)}")

# Add special tokens
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)

import json
with open('vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

Vocabulary size: 32


In [6]:
from transformers import Wav2Vec2CTCTokenizer, Wav2Vec2FeatureExtractor, Wav2Vec2Processor
from tqdm import tqdm
import numpy as np
from datasets import Dataset, DatasetDict

# Create the processor
tokenizer = Wav2Vec2CTCTokenizer("./vocab.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")
feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=False)
processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

print(f"Pad token id: {processor.tokenizer.pad_token_id}")
print(f"Vocabulary size: {len(processor.tokenizer.get_vocab())}")

# Create a new processed dataset
processed_datasets = DatasetDict()

# Process each split sequentially
for split in dataset:
    print(f"Processing {split} split...")
    processed_examples = []
    
    for i, example in enumerate(tqdm(dataset[split])):
        try:
            audio = example["audio"]
            
            # Process audio
            input_values = processor(audio["array"], sampling_rate=audio["sampling_rate"]).input_values[0]
            input_length = len(input_values)
            
            # Process text
            with processor.as_target_processor():
                labels = processor(example["text"]).input_ids
            
            # Create a new example with the processed data
            processed_example = {
                "input_values": input_values,
                "input_length": input_length,
                "labels": labels,
            }
            processed_examples.append(processed_example)
            
        except Exception as e:
            print(f"Error processing example {i} in {split}: {e}")
    
    # Convert list of dictionaries to dataset
    processed_datasets[split] = Dataset.from_pandas(pd.DataFrame(processed_examples))

# Replace the original dataset
dataset = processed_datasets

# Filter out examples that are too long 
max_input_length_in_sec = 10.0  # Adjust based on your data
if "input_length" in dataset["train"].column_names:
    dataset["train"] = dataset["train"].filter(
        lambda x: x < max_input_length_in_sec * processor.feature_extractor.sampling_rate, 
        input_columns=["input_length"]
    )

Pad token id: 33
Vocabulary size: 34
Processing train split...


100%|██████████| 80/80 [00:13<00:00,  5.85it/s]


Processing validation split...


100%|██████████| 10/10 [00:00<00:00, 18.30it/s]


Processing test split...


100%|██████████| 10/10 [00:00<00:00, 21.98it/s]


  0%|          | 0/1 [00:00<?, ?ba/s]

## Training & Evaluation

In [7]:
# Define data_collator before using it
from dataclasses import dataclass
from typing import Dict, List, Union
import torch

@dataclass
class DataCollatorCTCWithPadding:
    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        
        # Make sure labels only contain valid indices
        label_features = []
        for feature in features:
            # Ensure all label values are valid (below vocab size)
            labels = [l for l in feature["labels"] if l < len(processor.tokenizer)]
            if not labels:  # If empty after filtering, add pad token
                labels = [processor.tokenizer.pad_token_id]
            label_features.append({"input_ids": labels})

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            return_tensors="pt",
        )
        
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly (except for blank/pad token)
        labels = labels_batch["input_ids"].masked_fill(
            labels_batch.attention_mask.ne(1) & (labels_batch["input_ids"] != processor.tokenizer.pad_token_id), 
            -100
        )
        batch["labels"] = labels
        return batch

data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [8]:
# Define compute_metrics function
import numpy as np
from datasets import load_metric

wer_metric = load_metric("wer")

def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)
    return {"wer": wer}

In [16]:
output_dir = os.path.abspath(os.path.join("..", "..", "output", "models", "wav2vec2"))
best_model_dir = os.path.abspath("../../output/models/wav2vec2/best_model")

# Make sure the output directory exists
os.makedirs(output_dir, exist_ok=True)
os.makedirs(best_model_dir, exist_ok=True)

from transformers import Wav2Vec2ForCTC, TrainingArguments, Trainer

model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-base",
    ctc_loss_reduction="mean", 
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer)
)

# Double-check that the model's config matches your tokenizer
print(f"Model config pad_token_id: {model.config.pad_token_id}")
print(f"Model config vocab_size: {model.config.vocab_size}")

# If needed, manually set the CTC blank index to match pad_token_id
if hasattr(model, 'config'):
    model.config.blank_index = processor.tokenizer.pad_token_id

model.freeze_feature_encoder()  # Freeze the feature encoder for more efficient training

loading configuration file https://huggingface.co/facebook/wav2vec2-base/resolve/main/config.json from cache at C:\Users\mh/.cache\huggingface\transformers\c7746642f045322fd01afa31271dd490e677ea11999e68660a92619ec7c892b4.ce1f96bfaf3d7475cb8187b9668c7f19437ade45fb9ceb78d2b06a2cec198015
e:\Singaporean_English_ASR\.venv\lib\site-packages\transformers\configuration_utils.py:356: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(
Model config Wav2Vec2Config {
  "activation_dropout": 0.0,
  "adapter_kernel_size": 3,
  "adapter_stride": 2,
  "add_adapter": false,
  "apply_spec_augment": true,
  "architectures": [
    "Wav2Vec2ForPreTraining"
  ],
  "attention_dropout": 0.1,
  "bos_token_id": 1,
  "classifier_proj_size": 256,
  "codevector_dim": 2

Model config pad_token_id: 33
Model config vocab_size: 34


In [18]:
import pandas as pd
from IPython.display import display
from transformers import TrainerCallback 


# Create a custom callback to track metrics
class MetricsCallback(TrainerCallback):
    def __init__(self):
        self.metrics = {"epoch": [], "train_loss": [], "eval_loss": [], "eval_wer": []}
        
    def on_evaluate(self, args, state, control, metrics, **kwargs):
        # Save the evaluation metrics
        if "eval_loss" in metrics:
            self.metrics["eval_loss"].append(metrics["eval_loss"])
            self.metrics["eval_wer"].append(metrics["eval_wer"])
            self.metrics["epoch"].append(state.epoch)
            
            # If we have train loss from the last log, record it too
            if hasattr(state, "log_history") and len(state.log_history) > 0:
                for entry in reversed(state.log_history):
                    if "loss" in entry:
                        self.metrics["train_loss"].append(entry["loss"])
                        break
                else:
                    # If no train loss found, use NaN
                    self.metrics["train_loss"].append(float("nan"))
            
            # Display current metrics as a table
            metrics_df = pd.DataFrame(self.metrics)
            display(metrics_df)
            
            # Save the metrics to CSV
            metrics_df.to_csv(os.path.join(output_dir, "training_metrics.csv"), index=False)

# Create the metrics callback
metrics_callback = MetricsCallback()

In [19]:
training_args = TrainingArguments(
    output_dir=output_dir,
    group_by_length=True,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    num_train_epochs=15,
    fp16=True,
    gradient_checkpointing=True,
    learning_rate=1e-4,
    weight_decay=0.005,
    warmup_steps=1000,
    save_total_limit=3,  # Keep only the 3 best checkpoints
    push_to_hub=False,
    logging_dir=os.path.join(output_dir, "logs"),
    logging_steps=100,
    load_best_model_at_end=True,  # Load the best model when training ends
    metric_for_best_model="wer",  # Use WER for determining the best model
    greater_is_better=False,      # Lower WER is better
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [20]:
trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
    tokenizer=processor.feature_extractor,
    callbacks=[metrics_callback],  # Add our custom callback
)

Using amp half precision backend
e:\Singaporean_English_ASR\.venv\lib\site-packages\transformers\trainer.py:474: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler()


In [21]:
# Train the model
trainer.train()

The following columns in the training set  don't have a corresponding argument in `Wav2Vec2ForCTC.forward` and have been ignored: input_length. If input_length are not expected by `Wav2Vec2ForCTC.forward`,  you can safely ignore this message.
e:\Singaporean_English_ASR\.venv\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 79
  Num Epochs = 15
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 150


  0%|          | 0/150 [00:00<?, ?it/s]

e:\Singaporean_English_ASR\.venv\lib\site-packages\transformers\trainer.py:1949: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  ctx_manager = autocast(dtype=self.amp_dtype)
e:\Singaporean_English_ASR\.venv\lib\site-packages\torch\_dynamo\eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
The following columns in the evaluation set  don't have a corresponding argument in `Wav2Vec2ForCTC.forward` and have been ignored: input_length. If input_length are not expected by `Wav2Vec2ForCTC.forward`,  you can safely ignore this message.
***** Running Evalua

  0%|          | 0/2 [00:00<?, ?it/s]

{'eval_loss': 11.716703414916992, 'eval_wer': 1.0, 'eval_runtime': 0.8783, 'eval_samples_per_second': 11.386, 'eval_steps_per_second': 2.277, 'epoch': 1.0}


...

{'eval_loss': 1.2923663854599, 'eval_wer': 1.0, 'eval_runtime': 0.8082, 'eval_samples_per_second': 12.373, 'eval_steps_per_second': 2.475, 'epoch': 15.0}


,epoch,train_loss,eval_loss,eval_wer
0,1.0,NaN,11.716703,1.0
1,2.0,NaN,11.425539,1.0
2,3.0,NaN,10.662720,1.0
3,4.0,NaN,9.775192,1.0
4,5.0,NaN,8.321245,1.0
5,6.0,NaN,6.988673,1.0
6,7.0,NaN,4.790219,1.0
7,8.0,NaN,2.974931,1.0
8,9.0,NaN,1.920100,1.0
9,10.0,6.7177,1.607734,1.0


Saving model checkpoint to e:\Singaporean_English_ASR\output\models\wav2vec2\checkpoint-150
Configuration saved in e:\Singaporean_English_ASR\output\models\wav2vec2\checkpoint-150\config.json
Model weights saved in e:\Singaporean_English_ASR\output\models\wav2vec2\checkpoint-150\pytorch_model.bin
Feature extractor saved in e:\Singaporean_English_ASR\output\models\wav2vec2\checkpoint-150\preprocessor_config.json
Deleting older checkpoint [e:\Singaporean_English_ASR\output\models\wav2vec2\checkpoint-130] due to args.save_total_limit


Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from e:\Singaporean_English_ASR\output\models\wav2vec2\checkpoint-10 (score: 1.0).


{'train_runtime': 215.7622, 'train_samples_per_second': 5.492, 'train_steps_per_second': 0.695, 'train_loss': 5.103202921549479, 'epoch': 15.0}


TrainOutput(global_step=150, training_loss=5.103202921549479, metrics={'train_runtime': 215.7622, 'train_samples_per_second': 5.492, 'train_steps_per_second': 0.695, 'train_loss': 5.103202921549479, 'epoch': 15.0})

In [23]:
# After training, save the best model to the specified directory
trainer.save_model(best_model_dir)

# Also save the processor to the best model directory
processor.save_pretrained(best_model_dir)

print(f"Best model saved to {best_model_dir}")


Saving model checkpoint to e:\Singaporean_English_ASR\output\models\wav2vec2\best_model
Configuration saved in e:\Singaporean_English_ASR\output\models\wav2vec2\best_model\config.json
Model weights saved in e:\Singaporean_English_ASR\output\models\wav2vec2\best_model\pytorch_model.bin
Feature extractor saved in e:\Singaporean_English_ASR\output\models\wav2vec2\best_model\preprocessor_config.json
Feature extractor saved in e:\Singaporean_English_ASR\output\models\wav2vec2\best_model\preprocessor_config.json
tokenizer config file saved in e:\Singaporean_English_ASR\output\models\wav2vec2\best_model\tokenizer_config.json
Special tokens file saved in e:\Singaporean_English_ASR\output\models\wav2vec2\best_model\special_tokens_map.json


Best model saved to e:\Singaporean_English_ASR\output\models\wav2vec2\best_model


In [24]:
# After training, get the full metrics table
metrics_df = pd.DataFrame(metrics_callback.metrics)
print("\nFinal Training Metrics:")
print(metrics_df.to_string(index=False))

# You can also save the metrics to a CSV file
metrics_df.to_csv(os.path.join(output_dir, "final_training_metrics.csv"), index=False)


Final Training Metrics:
 epoch  train_loss  eval_loss  eval_wer
   1.0         NaN  11.716703       1.0
   2.0         NaN  11.425539       1.0
   3.0         NaN  10.662720       1.0
   4.0         NaN   9.775192       1.0
   5.0         NaN   8.321245       1.0
   6.0         NaN   6.988673       1.0
   7.0         NaN   4.790219       1.0
   8.0         NaN   2.974931       1.0
   9.0         NaN   1.920100       1.0
  10.0      6.7177   1.607734       1.0
  11.0      6.7177   1.498682       1.0
  12.0      6.7177   1.438490       1.0
  13.0      6.7177   1.393094       1.0
  14.0      6.7177   1.347303       1.0
  15.0      6.7177   1.292366       1.0


### Evaluation on Test Set

In [25]:
# Evaluate on test set
print("Evaluating on test set...")
eval_results = trainer.evaluate(dataset["test"])
print(f"Test WER: {eval_results['eval_wer']:.2f}%")

The following columns in the evaluation set  don't have a corresponding argument in `Wav2Vec2ForCTC.forward` and have been ignored: input_length. If input_length are not expected by `Wav2Vec2ForCTC.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 10
  Batch size = 8


Evaluating on test set...


e:\Singaporean_English_ASR\.venv\lib\site-packages\transformers\trainer.py:1949: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  ctx_manager = autocast(dtype=self.amp_dtype)


  0%|          | 0/2 [00:00<?, ?it/s]

,epoch,train_loss,eval_loss,eval_wer
0,1.0,NaN,11.716703,1.0
1,2.0,NaN,11.425539,1.0
2,3.0,NaN,10.662720,1.0
3,4.0,NaN,9.775192,1.0
4,5.0,NaN,8.321245,1.0
5,6.0,NaN,6.988673,1.0
6,7.0,NaN,4.790219,1.0
7,8.0,NaN,2.974931,1.0
8,9.0,NaN,1.920100,1.0
9,10.0,6.7177,1.607734,1.0


Test WER: 1.00%


## Hyperparameter Finetuning

In [ ]:
import os
import numpy as np
import pandas as pd
from dataclasses import dataclass
from typing import Dict, List, Union, Any
import torch
from datasets import load_metric
import optuna
from optuna.pruners import MedianPruner
from optuna.samplers import TPESampler
from transformers import (
    Wav2Vec2ForCTC,
    Wav2Vec2Processor,
    TrainingArguments,
    Trainer,
    EarlyStoppingCallback
)

# Define output directories
output_dir = os.path.abspath(os.path.join("..", "..", "output", "models", "wav2vec2"))
study_name = "wav2vec2_hyperparam_search"
study_storage = os.path.join(output_dir, f"{study_name}.db")
os.makedirs(output_dir, exist_ok=True)

# Load metric for evaluation
wer_metric = load_metric("wer")

# Data collator for CTC with padding
@dataclass
class DataCollatorCTCWithPadding:
    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # Split inputs and labels
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        # Pad inputs
        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            return_tensors="pt",
        )
        
        # Pad labels
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                return_tensors="pt",
            )

        # Replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)
        batch["labels"] = labels
        return batch

# Metric computation function
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    # Replace -100 with pad token id
    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    # Decode predictions and labels
    pred_str = processor.batch_decode(pred_ids)
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    # Compute WER
    wer = wer_metric.compute(predictions=pred_str, references=label_str)
    return {"wer": wer}

# Model initialization function
def create_model(trial):
    # Get hyperparameters from trial
    model = Wav2Vec2ForCTC.from_pretrained(
        "facebook/wav2vec2-base",
        ctc_loss_reduction="mean",
        pad_token_id=processor.tokenizer.pad_token_id,
    )
    
    # Freeze feature encoder based on trial suggestion
    if trial.suggest_categorical("freeze_feature_encoder", [True, False]):
        model.freeze_feature_encoder()
    
    # Freeze feature projection based on trial suggestion
    if trial.suggest_categorical("freeze_feature_projection", [True, False]):
        if hasattr(model, "wav2vec2") and hasattr(model.wav2vec2, "feature_projection"):
            for param in model.wav2vec2.feature_projection.parameters():
                param.requires_grad = False
    
    # Freeze a portion of transformer layers
    if hasattr(model, "wav2vec2") and hasattr(model.wav2vec2, "encoder"):
        num_layers = len(model.wav2vec2.encoder.layers)
        freeze_layers = trial.suggest_int("freeze_layers", 0, num_layers)
        if freeze_layers > 0:
            for layer in model.wav2vec2.encoder.layers[:freeze_layers]:
                for param in layer.parameters():
                    param.requires_grad = False
    
    return model

# Define the objective function for Optuna
def objective(trial):
    # Sample hyperparameters
    learning_rate = trial.suggest_float("learning_rate", 1e-5, 1e-3, log=True)
    weight_decay = trial.suggest_float("weight_decay", 1e-5, 1e-2, log=True)
    warmup_ratio = trial.suggest_float("warmup_ratio", 0.0, 0.2)
    attention_dropout = trial.suggest_float("attention_dropout", 0.0, 0.2)
    hidden_dropout = trial.suggest_float("hidden_dropout", 0.0, 0.2)
    batch_size = trial.suggest_categorical("batch_size", [4, 8, 16])
    
    # Calculate warmup steps based on ratio and total steps
    num_train_epochs = 10  # Reduced for hyperparameter search
    num_warmup_steps = int(len(dataset["train"]) // batch_size * num_train_epochs * warmup_ratio)
    
    # Set model configuration
    model = create_model(trial)
    
    # Set training arguments
    run_name = f"trial_{trial.number}"
    training_args = TrainingArguments(
        output_dir=os.path.join(output_dir, run_name),
        group_by_length=True,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        num_train_epochs=num_train_epochs,
        fp16=True,
        gradient_checkpointing=True,
        learning_rate=learning_rate,
        weight_decay=weight_decay,
        warmup_steps=num_warmup_steps,
        save_total_limit=1,
        push_to_hub=False,
        logging_dir=os.path.join(output_dir, run_name, "logs"),
        logging_steps=100,
        load_best_model_at_end=True,
        metric_for_best_model="wer",
        greater_is_better=False,
        report_to="none",
    )
    
    # Initialize data collator
    data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)
    
    # Setup trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=dataset["train"],
        eval_dataset=dataset["validation"],
        tokenizer=processor.feature_extractor,
        data_collator=data_collator,
        compute_metrics=compute_metrics,
        callbacks=[
            EarlyStoppingCallback(early_stopping_patience=3)
        ]
    )
    
    # Train and evaluate the model
    trainer.train()
    
    # Get validation WER
    eval_results = trainer.evaluate()
    
    # Record the best results
    trial.set_user_attr("best_wer", eval_results["eval_wer"])
    
    # Save trial results to CSV for easier analysis
    results_df = pd.DataFrame({
        "trial": [trial.number],
        "learning_rate": [learning_rate],
        "weight_decay": [weight_decay],
        "warmup_ratio": [warmup_ratio],
        "attention_dropout": [attention_dropout],
        "hidden_dropout": [hidden_dropout],
        "batch_size": [batch_size],
        "freeze_feature_encoder": [trial.params["freeze_feature_encoder"]],
        "freeze_feature_projection": [trial.params["freeze_feature_projection"]],
        "freeze_layers": [trial.params["freeze_layers"]],
        "wer": [eval_results["eval_wer"]]
    })
    
    # Append or create CSV file with results
    results_path = os.path.join(output_dir, "hyperparameter_search_results.csv")
    if os.path.exists(results_path):
        results_df.to_csv(results_path, mode='a', header=False, index=False)
    else:
        results_df.to_csv(results_path, index=False)
    
    # Return validation WER (lower is better)
    return eval_results["eval_wer"]

# Create an Optuna study
def run_hyperparameter_search(n_trials=20):
    # Create a study with the TPE sampler and median pruner
    study = optuna.create_study(
        study_name=study_name,
        storage=f"sqlite:///{study_storage}",
        load_if_exists=True,
        direction="minimize",  # Minimize WER
        sampler=TPESampler(seed=42),
        pruner=MedianPruner(n_startup_trials=5, n_warmup_steps=5)
    )
    
    # Run optimization
    study.optimize(objective, n_trials=n_trials)
    
    # Print study statistics
    print(f"\nNumber of finished trials: {len(study.trials)}")
    print("Best trial:")
    best_trial = study.best_trial
    print(f"  Value (WER): {best_trial.value:.4f}")
    print("  Params:")
    for key, value in best_trial.params.items():
        print(f"    {key}: {value}")
    
    # Get best parameters and train final model
    best_params = study.best_params
    
    # Create a results dataframe with all trials
    trials_df = pd.DataFrame([
        {
            "trial": t.number,
            **t.params,
            "wer": t.value,
            "state": t.state,
        }
        for t in study.trials if t.state.is_finished()
    ])
    
    # Sort by WER (ascending)
    trials_df = trials_df.sort_values("wer")
    
    # Save complete results
    trials_df.to_csv(os.path.join(output_dir, "all_trials_results.csv"), index=False)
    
    # Display top 5 results
    print("\nTop 5 configurations:")
    display(trials_df.head(5))
    
    # Create visualizations
    try:
        import matplotlib.pyplot as plt
        from optuna.visualization import plot_param_importances, plot_optimization_history
        
        # Parameter importance plot
        importance_fig = plot_param_importances(study)
        importance_fig.write_image(os.path.join(output_dir, "param_importance.png"))
        
        # Optimization history plot
        history_fig = plot_optimization_history(study)
        history_fig.write_image(os.path.join(output_dir, "optimization_history.png"))
        
    except ImportError:
        print("Skipping visualizations - matplotlib or plotly not available")
    
    return best_params

# Train final model with best hyperparameters
def train_final_model(best_params):
    print("\nTraining final model with best hyperparameters...")
    
    # Create model with best configuration
    class ModelCreator:
        def suggest_categorical(self, name, choices):
            return best_params[name]
        
        def suggest_int(self, name, low, high):
            return best_params[name]
    
    trial = ModelCreator()
    model = create_model(trial)
    
    # Calculate warmup steps based on best parameters
    num_train_epochs = 30  # Full training
    batch_size = best_params["batch_size"]
    warmup_ratio = best_params["warmup_ratio"]
    num_warmup_steps = int(len(dataset["train"]) // batch_size * num_train_epochs * warmup_ratio)
    
    # Set training arguments for final model
    final_output_dir = os.path.join(output_dir, "final_model")
    training_args = TrainingArguments(
        output_dir=final_output_dir,
        group_by_length=True,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        num_train_epochs=num_train_epochs,
        fp16=True,
        gradient_checkpointing=True,
        learning_rate=best_params["learning_rate"],
        weight_decay=best_params["weight_decay"],
        warmup_steps=num_warmup_steps,
        save_total_limit=2,
        push_to_hub=False,
        logging_dir=os.path.join(final_output_dir, "logs"),
        logging_steps=100,
        load_best_model_at_end=True,
        metric_for_best_model="wer",
        greater_is_better=False,
    )
    
    # Initialize data collator
    data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)
    
    # Setup trainer for final model
    final_trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=dataset["train"],
        eval_dataset=dataset["validation"],
        tokenizer=processor.feature_extractor,
        data_collator=data_collator,
        compute_metrics=compute_metrics,
    )
    
    # Train final model
    final_trainer.train()
    
    # Evaluate on test set if available
    if "test" in dataset:
        test_results = final_trainer.evaluate(dataset["test"])
        print(f"Test WER: {test_results['eval_wer']:.4f}")
    
    # Save final model and processor
    final_trainer.save_model(final_output_dir)
    processor.save_pretrained(final_output_dir)
    
    print(f"Final model saved to {final_output_dir}")
    
    return final_trainer

# Main execution
if __name__ == "__main__":
    # Load processor (ensure this matches your dataset preprocessing)
    processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base")
    
    # Assuming dataset is already loaded and preprocessed
    # Run hyperparameter search
    best_params = run_hyperparameter_search(n_trials=20)
    
    # Train final model with best hyperparameters
    final_trainer = train_final_model(best_params)